<a href="https://colab.research.google.com/github/MatheusABomfim/Mest_MBL2_ML_HTLV/blob/main/MEST_HTLV%2BMBL2%2BCLINICAL_Hyperparameters_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading data and packages**

**Libraries import**

In [1]:
# Import necessary libraries and packages
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

# sklearn imports
from sklearn.model_selection import (
    train_test_split, cross_val_score, LeaveOneOut, cross_validate,
    learning_curve
)

from sklearn.ensemble import (
    ExtraTreesClassifier, RandomForestClassifier,
    AdaBoostClassifier, GradientBoostingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay, classification_report,
    recall_score, accuracy_score, roc_curve, auc, make_scorer,
    precision_score, f1_score
)
from sklearn.preprocessing import (
    LabelEncoder, MinMaxScaler, MaxAbsScaler, QuantileTransformer,
    RobustScaler, StandardScaler
)
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

# imbalanced-learn imports
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN

# joblib imports
from joblib import dump, load

# scikit-optimize imports
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# statsmodels imports
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# openpyxl imports
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# ipywidgets imports
from ipywidgets import interact, widgets

# Ignore warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import make_scorer
import warnings
import os
from joblib import dump
from imblearn.over_sampling import RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler, SMOTENC
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from sklearn.model_selection import train_test_split
from ipywidgets import widgets, interact


# Data selection
clinic_dataset = '../datasets/df_clinic_ENTPD1.csv'
hemogram_dataset = '../datasets/df_hemogram_ENTPD1.csv'
clinic_dataset_cat1 = '../datasets/cat1_df_clinic_ENTPD1.csv'
hemogram_dataset_cat1 = '../datasets/df_hemogram_cat_1.csv'
clinic_dataset_cat2 = '../datasets/cat2_df_clinic_ENTPD1.csv'
hemogram_dataset_cat2 = '../datasets/cat2_df_hemogram_ENTDP1.csv'

**Data import**

In [ ]:
# Defining global variables to store the DataFrame
df = pd.DataFrame()
dataset_name = None
X = None
y = None
X_train = None
X_test = None
y_train = None
y_test = None
balancing_method_selected = None

# Function to list CSV files in the directory
def list_csv_files(directory):
    return [f for f in os.listdir(directory) if f.endswith('.csv')]

# Function to load and process the selected dataset with STRATIFIED split
def load_and_process_data(selected_file):
    global df, dataset_name, X, y, X_train, X_test, y_train, y_test
    dataset_name = selected_file
    file_path = os.path.join('..', 'datasets', 'feature_selection', dataset_name)

    if not os.path.exists(file_path):
        print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
        return None

    df = pd.read_csv(file_path)

    # Converting 'Gender' to numeric values
    if 'Sexo' in df.columns:
        df['Sexo'] = df['Sexo'].replace({'M': 0, 'F': 1})

    # Normalizing 'Age_clinic'
    if 'Idade_clinica' in df.columns:
        scaler = MinMaxScaler()
        df['Idade_clinica'] = scaler.fit_transform(df[['Idade_clinica']])

    # Defining X (features) and y (target)
    X = df.drop(columns=['P-HAM/TSP']) 
    y = df['P-HAM/TSP'] 
    
    # Verificar a distribuição de classes original
    print(f"Distribuição original de classes:")
    print(f"Classe 0: {(y == 0).sum()} amostras ({((y == 0).sum()/len(y))*100:.1f}%)")
    print(f"Classe 1: {(y == 1).sum()} amostras ({((y == 1).sum()/len(y))*100:.1f}%)")
    
    # Stratified split 70% for training and 30% for testing
    # Garantindo que 70% das amostras de CADA CLASSE vão para treino
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.3, 
        stratify=y,  # ← ESTA É A CHAVE: estratificação por classe
        random_state=42
    )
    
    print(f"\nData Loaded and Processed Successfully! Dataset: {dataset_name}")
    print(f"Training set shape: {X_train.shape}")
    print(f"Test set shape: {X_test.shape}")
    
    # Verificar a distribuição de classes nos conjuntos de treino e teste
    print(f"\nDistribuição de classes no conjunto de TREINO:")
    print(f"Classe 0: {(y_train == 0).sum()} amostras ({((y_train == 0).sum()/len(y_train))*100:.1f}%)")
    print(f"Classe 1: {(y_train == 1).sum()} amostras ({((y_train == 1).sum()/len(y_train))*100:.1f}%)")
    
    print(f"\nDistribuição de classes no conjunto de TESTE:")
    print(f"Classe 0: {(y_test == 0).sum()} amostras ({((y_test == 0).sum()/len(y_test))*100:.1f}%)")
    print(f"Classe 1: {(y_test == 1).sum()} amostras ({((y_test == 1).sum()/len(y_test))*100:.1f}%)")
    
    print(f"\n✅ Conjunto de teste (30%) foi isolado automaticamente!")
    print(f"✅ Balanceamento será aplicado apenas nos dados de treino (70%)!")
    
    return df.head()

# Function to define the selected balancing method
def set_balancing_method(method):
    global balancing_method_selected  
    balancing_method_selected = method  
    print(f"Método de balanceamento selecionado: {balancing_method_selected}")

# Function to apply the selected balancing method ONLY to training data
def apply_balancing():
    global balancing_method_selected, X_train, y_train
    
    if balancing_method_selected is None:
        print("Nenhum método de balanceamento foi selecionado.")
        return X_train, y_train

    if balancing_method_selected == 'Random Over Sampler':
        balancer = RandomOverSampler(random_state=42)
    elif balancing_method_selected == 'SMOTENC':
        # Identify categorical columns
        categorical_features = [i for i, col in enumerate(X_train.columns) if col != 'Idade_clinica']
        balancer = SMOTENC(categorical_features=categorical_features, random_state=42)
    elif balancing_method_selected == 'Random Under Sampler':
        balancer = RandomUnderSampler(random_state=42)
    elif balancing_method_selected == 'Tomek Links':
        balancer = TomekLinks()
    else:
        print(f"Método de balanceamento desconhecido: {balancing_method_selected}")
        return X_train, y_train

    # Applying the balancing method ONLY to training data
    X_res, y_res = balancer.fit_resample(X_train, y_train)
    
    print(f"Shape dos dados de treino balanceados: {X_res.shape}, {y_res.shape}")
    print(f"Distribuição de classes após balanceamento: {np.bincount(y_res)}")
    print(f"✅ Balanceamento aplicado APENAS no conjunto de treino!")
    print(f"❌ Conjunto de teste permanece intacto com distribuição original!")
    
    return X_res, y_res

# Define the directory where the CSV files are located
directory = '../datasets/feature_selection/'

# List the CSV files available in the directory
csv_files = list_csv_files(directory)

# Create widgets
file_selector = widgets.Dropdown(
    options=csv_files,
    description='Select File:',
    style={'description_width': 'initial'}
)

balancing_methods_list = ['Random Over Sampler', 'SMOTENC', 'Random Under Sampler', 'Tomek Links']
method_selector = widgets.Dropdown(
    options=balancing_methods_list,
    description='Balancing Method:',
    style={'description_width': 'initial'}
)

# Interactive functions
@interact(selected_file=file_selector)
def load_and_process_selected_file(selected_file):
    print("="*60)
    print("CARREGANDO E PROCESSANDO DATASET")
    print("="*60)
    result = load_and_process_data(selected_file)
    print("="*60)
    return result

@interact(selected_method=method_selector)
def apply_selected_balancing_method(selected_method):
    print("="*60)
    print("APLICANDO BALANCEAMENTO (APENAS TREINO)")
    print("="*60)
    set_balancing_method(selected_method)
    result = apply_balancing()
    print("="*60)
    return result


interactive(children=(Dropdown(description='Select File:', options=('df_Clinic_dataset_cat_3_RUS.csv', 'cat2_d…

interactive(children=(Dropdown(description='Balancing Method:', options=('Random Over Sampler', 'SMOTENC', 'Ra…

In [18]:
print(y_test)

177    0
39     0
196    0
228    0
81     1
      ..
254    0
16     0
161    0
74     0
68     0
Name: P-HAM/TSP, Length: 78, dtype: int64


# Hyperparametes tuning by bayeasian optimization

**Search for the best hyperparameters and saved definitions**

In [ ]:
# Define a random state for reproducibility
RANDOM_STATE = 42

# Ignore warnings
warnings.filterwarnings('ignore')

# Common settings for all models
common_settings = {
    'cv': LeaveOneOut(),
    'n_iter': 10,
    'n_jobs': -1,
    'verbose': 0,
}

# Bayesian search function
def bayes_search(model, space):
    bs = BayesSearchCV(
        model, space,
        n_iter=common_settings['n_iter'],
        cv=common_settings['cv'],
        random_state=RANDOM_STATE,
        scoring=make_scorer(geometric_mean_score)
    )
    return bs

# Model space and hyperparameters
model_space = {
    'DT': {
        'model': DecisionTreeClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__max_depth': Integer(3, 5),
            'classifier__min_samples_split': Integer(5, 10),
            'classifier__min_samples_leaf': Integer(1, 5),
            'classifier__criterion': Categorical(['gini', 'entropy']),
            'classifier__ccp_alpha': Real(0.0, 0.1),
        }
    },
    'GB': {
        'model': GradientBoostingClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__n_estimators': Integer(50, 500),
            'classifier__learning_rate': Real(0.01, 1.0, 'log-uniform'),
            'classifier__max_depth': Integer(1, 10),
            'classifier__min_samples_split': Integer(2, 20),
            'classifier__min_samples_leaf': Integer(1, 20),
            'classifier__subsample': Real(0.5, 1.0, 'log-uniform'),
            'classifier__loss': Categorical(['exponential', 'log_loss']),
        }
    },
    'LR': {
        'model': LogisticRegression(random_state=RANDOM_STATE),
        'space': {
            'classifier__C': Real(0.0001, 1000, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__solver': Categorical(['liblinear', 'sag', 'saga']),
            'classifier__class_weight': Categorical(['balanced', None]),
            'classifier__tol': Real(0.0001, 0.001),
        }
    },
    'MLP': {
        'model': MLPClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__hidden_layer_sizes': Integer(2, 16),
            'classifier__activation': Categorical(['logistic', 'tanh', 'relu']),
            'classifier__learning_rate': Categorical(['constant', 'adaptive']),
            'classifier__learning_rate_init': Real(0.001, 0.1, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__solver': Categorical(['sgd', 'adam']),
            'classifier__momentum': Real(0.1, 0.9, 'log-uniform'),
        }
    },
    'NB': {
        'model': GaussianNB(),
        'space': None,  # GaussianNB não possui hiperparâmetros para ajuste
    },
    'RF': {
        'model': RandomForestClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__n_estimators': Integer(50, 500),
            'classifier__max_depth': Integer(3, 10),
            'classifier__min_samples_split': Integer(2, 10),
            'classifier__min_samples_leaf': Integer(1, 5),
            'classifier__criterion': Categorical(['gini', 'entropy']),
            'classifier__max_samples': Real(0.5, 1.0, 'log-uniform'),
            'classifier__class_weight': Categorical(['balanced', 'balanced_subsample']),
        }
    },
    'SVM': {
        'model': SVC(probability=True, random_state=RANDOM_STATE),
        'space': {
            'classifier__kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
            'classifier__gamma': Real(0.001, 1000),
            'classifier__degree': Integer(2, 5),
            'classifier__coef0': Real(0.0001, 1, 'log-uniform'),
            'classifier__C': Real(0.001, 1000),
            'classifier__tol': Real(0.00001, 0.1, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__class_weight': Categorical([None, 'balanced']),
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'space': {
            'classifier__n_neighbors': Integer(1, 20),
            'classifier__weights': Categorical(['uniform', 'distance']),
            'classifier__algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
            'classifier__leaf_size': Integer(20, 50),
            'classifier__p': Integer(1, 2),
        }
    },
    'AB': {
        'model': AdaBoostClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__n_estimators': Integer(50, 500),
            'classifier__learning_rate': Real(0.01, 1.0, 'log-uniform'),
        }
    },
    'LSVM': {
        'model': LinearSVC(random_state=RANDOM_STATE),
        'space': {
            'classifier__C': Real(0.0001, 1000, 'log-uniform'),
            'classifier__tol': Real(0.00001, 0.1, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__class_weight': Categorical([None, 'balanced']),
        }
    },
}


# Template saving directory
save_dir = '../models/'

# Define the balancing methods
balancing_methods = {
    'Random Over Sampler': RandomOverSampler(random_state=RANDOM_STATE),
    'SMOTENC': None,
    'Random Under Sampler': RandomUnderSampler(random_state=RANDOM_STATE),
    'Tomek Links': TomekLinks()
}

print(f"Starting training with Leave-One-Out on 70% training set")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples (completely isolated)")
print(f"LOO will perform {X_train.shape[0]} iterations\n")

# Iterating over the models in the search space
for model_name in model_space:
    model = model_space[model_name]['model']
    space = model_space[model_name]['space']

    print(f"\n{'='*50}")
    print(f"Training model: {model_name}")
    print(f"{'='*50}")

    # Confirmation of the selected balancing method
    print(f"Balancing method selected: {balancing_method_selected}")

    # Verification for SMOTENC: adjust categorical columns if necessary
    if balancing_method_selected == 'SMOTENC':
        categorical_columns = [col for col in X_train.columns if col != 'Idade_clinica']
        categorical_features = [X_train.columns.get_loc(col) for col in categorical_columns]
        balancing_methods['SMOTENC'] = SMOTENC(categorical_features=categorical_features, random_state=RANDOM_STATE)

    # Define the pipeline with the selected balancing method
    pipeline = Pipeline([
        ('normalization', MinMaxScaler()),
        ('balancing', balancing_methods[balancing_method_selected]),
        ('classifier', model)
    ])

    # Check that the search space is not empty
    if space:
        bs = bayes_search(pipeline, space)
        
        # Training with just the 70% TRAINING data using LOO
        print(f"Performing Bayesian search with Leave-One-Out on {X_train.shape[0]} samples...")
        bs.fit(X_train, y_train)  

        print(f'Best G-mean value (LOO cross-validation): {bs.best_score_:.4f}')
        print(f'Best parameters: {bs.best_params_}')

        # Evaluation on the training set
        y_train_pred = bs.best_estimator_.predict(X_train)
        train_gmean = geometric_mean_score(y_train, y_train_pred)
        print(f'Train G-mean: {train_gmean:.4f}')

        # Saving the best model
        model_path = os.path.join(save_dir, f'best_{model_name}_model.joblib')
        dump(bs.best_estimator_, model_path)
        print(f'Model saved to: {model_path}')

    else:
        # Train the model directly if there are no hyperparameters to adjust
        pipeline.fit(X_train, y_train)  
        print('Model trained directly without Bayesian search.')

        # Avaliação no conjunto de treino
        y_train_pred = pipeline.predict(X_train)
        train_gmean = geometric_mean_score(y_train, y_train_pred)
        print(f'Train G-mean: {train_gmean:.4f}')

        # Saving the trained model
        model_path = os.path.join(save_dir, f'best_{model_name}_model.joblib')
        dump(pipeline, model_path)
        print(f'Model saved to: {model_path}')

print(f"\n{'='*50}")
print("TRAINING COMPLETED WITH LEAVE-ONE-OUT!")
print(f"All models trained on {X_train.shape[0]} samples (70%) using LOO")
print(f"Test set with {X_test.shape[0]} samples (30%) remains completely isolated")
print("Use the test set for final external validation.")
print(f"{'='*50}")


Starting training with Leave-One-Out on 70% training set
Training set size: 180 samples
Test set size: 78 samples (completely isolated)
LOO will perform 180 iterations


Training model: DT
Balancing method selected: Random Over Sampler
Performing Bayesian search with Leave-One-Out on 180 samples...
Best G-mean value (LOO cross-validation): 0.8667
Best parameters: OrderedDict([('classifier__ccp_alpha', 0.04101039588533139), ('classifier__criterion', 'entropy'), ('classifier__max_depth', 5), ('classifier__min_samples_leaf', 2), ('classifier__min_samples_split', 8)])
Train G-mean: 0.8144
Model saved to: ../models/best_DT_model.joblib

Training model: GB
Balancing method selected: Random Over Sampler
Performing Bayesian search with Leave-One-Out on 180 samples...
Best G-mean value (LOO cross-validation): 0.8111
Best parameters: OrderedDict([('classifier__learning_rate', 0.12212543829922852), ('classifier__loss', 'log_loss'), ('classifier__max_depth', 5), ('classifier__min_samples_leaf', 17